# Knowledge Agent with Azure AI Agent Service

## Install libraries

In [ ]:
%pip install azure-ai-projects azure-identity azure-search-documents

## Connect to Azure AI Foundry project

In [ ]:
import os
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.search.documents  import SearchClient

# Get connection string for project
project_connection_string = "<your_project_connection_string>"

# Create a project client using environment variables loaded from the .env file
project_client = AIProjectClient.from_connection_string(
    conn_str=project_connection_string, credential=DefaultAzureCredential()
)

# Create chat and embeddings clients using the project client
chat_client = project_client.inference.get_chat_completions_client()
embeddings_client = project_client.inference.get_embeddings_client()

# Use the project client to get the default search connection
# Before you need to create a connection in the Azure AI Foundry portal
search_connection = project_client.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH, include_credentials=True
)

# Create a search client using the search connection
search_client = SearchClient(
    index_name="docs",
    endpoint=search_connection.endpoint_url,
    credential=AzureKeyCredential(key=search_connection.key),
)

## (Optional) Enable tracing

In [ ]:
# Make sure you have created an Application Insights resource for your Azure AI Foundry hub before running this code.
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry import trace

tracer = trace.get_tracer(__name__)

os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"
application_insights_connection_string = project_client.telemetry.get_connection_string()
configure_azure_monitor(connection_string=application_insights_connection_string)

## Use RAG function as a tool in function calling

In [ ]:
import json
from azure.ai.inference.prompts import PromptTemplate
from azure.search.documents.models import QueryAnswerType, QueryCaptionType, QueryType, VectorizedQuery
from typing import Any, Callable, Set

#@tracer.start_as_current_span(name="get_search_query")  # Uncomment this line to enable tracing for this function
# Define a function to generate a search query
def get_search_query(user_question: str):
    prompt_template_str = """
    system:
    # Instructions
    - You are an AI assistant.
    - Given the user's question, respond with a search query that can be used to retrieve relevant documents for the user's question based on the intent.
    - Be specific in what the user is asking about.
    - Provide only the search query in the response.

    # Example
    With a user query like below:
    "What was the total revenue in 2024?"

    Respond with:
    "total revenue in 2024"

    user:
    Return the search query for the following user question:
    {{user_question}}
    """

    prompt_template = PromptTemplate.from_string(prompt_template=prompt_template_str)

    messages = prompt_template.create_messages(user_question=user_question)

    response = chat_client.complete(
        model="gpt-4o",
        messages=messages,
        temperature=0.1
    )

    search_query = response.choices[0].message.content

    return search_query

#@tracer.start_as_current_span(name="get_documents")  # Uncomment this line to enable tracing for this function
# Define a function that retrieves documents from the search index based on a search query
def get_documents(search_query: str):
    # Embed the search query
    embedding = embeddings_client.embed(model="text-embedding-3-large", input=search_query)
    search_vector = embedding.data[0].embedding

    # Search the index for document chunks matching the search query
    vector_query = VectorizedQuery(
        vector=search_vector,
        k_nearest_neighbors=50,
        fields="content_vector"
    )

    search_results = search_client.search(
        search_text=search_query,
        vector_queries=[vector_query],
        query_type=QueryType.SEMANTIC, semantic_configuration_name="default", query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
        top=5,
        select=["id", "page", "base64_image", "content"]
    )

    documents = [
        {
            "id": result["id"],
            "page": result["page"],
            "base64_image": result["base64_image"],
            "content": result["content"]
        }
        for result in search_results
    ]

    return documents

#@tracer.start_as_current_span(name="get_ansewr_from_documents")  # Uncomment this line to enable tracing for this function
# Define a function that generates an answer with documents
def get_answer_from_documents(user_question: str, documents: list):
    system_prompt = """
    - You are an expert helping employees from AVL to find information in the knowledge base of AVL.
    - You are given a user question and a set of text descriptions and screenshots of slides from a presentation.
    - Use the text descriptions and screenshots as context to answer the questions as completely, correctly, and concisely as possible.
    - Not all documents are relevant to the question, so only use the relevant documents to answer the question.
    - Don't try to make up any answers. If the answer cannot be retrieved from the context and you do not know the answer, then answer 'Sorry, I do not know.'.
    - Add sources to the answer listing the page numbers you used and that are relevant to answer the question.
    - The final response must be in JSON format with two fields:
        - answer: The generated answer to the user's question.
        - sources: A list of page numbers for each cited source
    - Do not use ```json```

    Here is an example of the final response:
    {
        "answer": "The total revenue in 2024 was $245,122 million."
        "sources": [1]
    }
    """

    context = "\n\n".join(f"Document ID: {document['id']}\nPage: {document['page']}\nContent:\n{document['content']}" for document in documents)
    user_prompt = f"""User: {user_question}\n\n
    Answer the user's question based on the following context:\n\nDocuments:\n\n{context}"""
    user_content = [
        {
            "type": "text",
            "text": user_prompt,
        }
    ]
    images = [{ "type": "image_url", "image_url": {"url": f"data:image/png;base64,{document['base64_image']}"} } for document in documents]
    user_content += images

    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_content,
        }
    ]

    response = chat_client.complete(
        model="gpt-4o",
        messages=messages,
        temperature=0.1
    )

    result_str = response.choices[0].message.content
    result_json = json.loads(result_str)

    return result_json

#@tracer.start_as_current_span(name="rag")  # Uncomment this line to enable tracing for this function
def rag(user_question: str):
    """
    Search the knowledge base and generate an answer to the user's question with the retrieved documents.

    :param user_question (str): The original user question.
    :return: Answer to the user's question based on the retrieved documents.
    :rtype: str
    """
    #span = trace.get_current_span()  # Uncomment this line to enable tracing

    # Generate search query
    search_query = get_search_query(user_question=user_question)
    #span.set_attribute("search_query", search_query)  # Uncomment this line to enable tracing

    # Retrieve documents with search query
    documents = get_documents(search_query=search_query)

    # Generate answer based on retrieved documents
    answer = get_answer_from_documents(user_question=user_question, documents=documents)
    #span.set_attribute("answer", answer["answer"])  # Uncomment this line to enable tracing
    #span.set_attribute("sources", answer["sources"])  # Uncomment this line to enable tracing
    
    return answer["answer"]

user_functions: Set[Callable[..., Any]] = {
    rag
}

In [ ]:
from azure.ai.projects.models import FunctionTool, ToolSet

# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

In [ ]:
functions.definitions

## Create an agent

In [ ]:
instructions = """You are an AI assistant for AVL employees helping answering user questions.
Use your knowledge base to answer the user questions with the context retrieved from documents.
"""

# Create agent with toolset and process a run
agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="avl-knowledge-agent",
    instructions=instructions,
    toolset=toolset
)
print(f"Created agent, ID: {agent.id}")

In [ ]:
# Helper functions to display answer from agent
from IPython.display import display, Image, Markdown

def download_and_save_image(image_file_id: str, image_file_name: str) -> None:
    project_client.agents.save_file(file_id=image_file_id, file_name=image_file_name)


def pretty_print(message):
    role_label = "User" if message.role == "user" else "Assistant"
    # Check the type of message content and handle accordingly
    for content in message.content:
        if content.type == "text":
            message_content = content.text.value
            display(Markdown(f"**{role_label}**: {message_content}\n"))
            if content.text.annotations:
                display(Markdown("Sources:"))
                for annotation in content.text.annotations:
                    if annotation.type == "url_citation":
                        display(Markdown(f"{annotation.text}\n* Title: {annotation['url_citation']['title']}\n* URL: {annotation['url_citation']['url']}"))
        elif content.type == "image_file":
            # Handle image file content, e.g., print the file ID or download the image
            image_file_id = content.image_file.file_id
            # Define a path to save the image
            image_file_name = f"image_{image_file_id}.png"
            # Download and save the image
            download_and_save_image(image_file_id, image_file_name)
            # Display the image within Jupyter Notebook
            display(Markdown(f"**{role_label}**: Image {image_file_id} generated"))
            display(Image(filename=image_file_name))

## Create a thread

In [ ]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

## Create a message and process agent run

In [ ]:
# User prompt
user_prompt = "The customer complained about an issue with a noisy E-Axle. What can I offer?"

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=user_prompt,
)
display(Markdown(f"**User**: {user_prompt}\n"))

# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)

# Check if run has failed
if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

# Fetch messages
messages = project_client.agents.list_messages(thread_id=thread.id)
agent_message = messages.data[0]
pretty_print(agent_message)

## Retrieve run steps

In [ ]:
# Retrieve run steps
run_steps = project_client.agents.list_run_steps(thread_id=thread.id, run_id=run.id)
for step in run_steps.data:
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            if tool_call.type == "code_interpreter":
                print("Used Code Interpreter with:")
                print(tool_call.code_interpreter.input)
            elif tool_call.type == "bing_grounding":
                print("Used Bing Grounding tool")
            elif tool_call.type == "function":
                print(f"Used function {tool_call.function.name} with arguments:")
                print(tool_call.function.arguments)
            print("-------------------------------------------------------------------------------")

## Display conversation history

In [ ]:
# Print conversation
for message in reversed(messages.data):
    pretty_print(message)

## Delete agent

In [ ]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")